# Semantic segmentation

<img src='imgs/semantic_segmentation.jpg' width=600 />

# Бородатые методы
<img src='imgs/PaiMeiPromo.jpg' width=300 />

Общий принцип:
```
#include <opencv2/imgproc/imgproc.hpp>
```

Техники:
* Просто `threshold`
* Не просто `threshold` -- `otsu threshold`
* Сегментация на основе граней, ребер и т.д.
* Сегментация через кластеризацию


Два примера из `sklearn` и `scikit-image`:
* https://scikit-learn.org/stable/auto_examples/cluster/plot_coin_segmentation.html
* https://scikit-image.org/docs/dev/user_guide/tutorial_segmentation.html

# https://scikit-learn.org/stable/auto_examples/cluster/plot_coin_segmentation.html

Спектральная кластеризация.  
  
# https://scikit-image.org/docs/dev/user_guide/tutorial_segmentation.html
* подбор порога для ч/б картинки
* детекция по ребрам
    * детектор ребер Canny
    * заполнение "дыр"
* детекция по регионам
    * оператор Собеля
    * watershed

# Оператор Собеля
Две свертки:

$$\LARGE G_1 =  \begin{bmatrix} -1&0&+1\\ -2&0&+2\\-1&0&+1 \end{bmatrix}$$

$$\LARGE G_2 =  \begin{bmatrix} -1&-2&-1\\ 0&0&0\\+1&+2&+1 \end{bmatrix}$$

Собель - оригинал и результат
<table><tr>
<td><img src='imgs/sobel_original.png' width=300 /></td> 
<td><img src='imgs/sobel.png' width=300 /></td>
</tr></table>

# Watershed

Семейство алгоритмов, заливающих изображение по принципам FloodFill

Пример - алгоритм Майера(ну примерно так):
* Выбираем стартовые точки
* От каждой стартовой точки делаем заливку пока не упремся слишком сильное изменение тона изображения


* Есть множество других идей как делать то же самое
* Есть алгоритмы на основе глубоких сетей(https://arxiv.org/pdf/1611.08303.pdf)

# Современные методы

Попробуем скользящее окно - для каждого участка будем классифицировать его(допустим) центр.
<img src='imgs/fcn_rolling_window.jpg' width=800 />

Это неэффективно -- мы не переиспользуем данные с перекрывающихся частей изображения.

# Сделаем предсказания сразу же со всего изображения

<img src='imgs/fcn2.jpg' width=800 />
(конечно, много сверток будет трудно считать)

<img src='imgs/fcn3.jpg' width=800 />

# Upsampling

<img src='imgs/upsampl1.jpg' width=800 />


<table><tr>
<td><img src='imgs/ups2.jpg' width=600 /></td> 
<td><img src='imgs/ups3.jpg' width=600 /></td>
</tr></table>

## Короткий список обычных интерполяций:
* Bilinear
* Bicubic
* Spline
* Sinc
* Lanczos
* много других

<img src='imgs/interpolations.png'/>


# Transpose Convolution

<center><h1> 3 x 3 transpose convolution, stride 2 pad 1</h1></center>
<img src='imgs/transp_c1.jpg' width=600 />

<img src='imgs/ups4.jpg' width=600 />

* Фильтр сдвигается на 2 пикселя для каждого пикселя на входе
* stride - отношение между выходом и входом

# Transpose Convolution
Одномерный случай

<img src='imgs/1d_conv.png'/>

### Свертка(обычная) как умножение матриц

$$ \LARGE \vec{x} * \vec{a} = X \vec{a}$$

$$ \LARGE \begin{bmatrix} x& y & z& 0 &0 &0 \\0 & x& y & z& 0 &0 \\ 0& 0& x& y & z& 0  \\ 0& 0 &0& x& y & z  \end{bmatrix} *\begin{bmatrix} 0 \\ a\\b\\c\\d\\0 \end{bmatrix}= \begin{bmatrix} ay+bz \\ ax+by+cz\\bx+cy+dz\\cx+dy \end{bmatrix} $$

### И транспонированная
$$ \LARGE \vec{x}^T * \vec{a} = X^T \vec{a}$$
    
$$ \LARGE \begin{bmatrix} x&0 &0&0 \\y& x & 0 &0 \\ z& y& x& 0  \\ 0& z &y& x \\ 0&0&z&y\\0&0&0&z  \end{bmatrix} *\begin{bmatrix}a\\b\\c\\d \end{bmatrix}= \begin{bmatrix} ax \\ ay+bx\\az+by+cx\\bz+cy+dx\\cz+dy\\dz \end{bmatrix} $$

# FCN

<img src='imgs/fcn1.jpg'/>

https://github.com/aurora95/Keras-FCN

Один блок в FCN
```
x = Conv2D(64, (3, 3), activation='relu', 
    padding='same', 
    name='block1_conv1', 
    kernel_regularizer=l2(weight_decay))(img_input)
x = Conv2D(64, (3, 3), activation='relu', 
    padding='same', 
    name='block1_conv2', 
    kernel_regularizer=l2(weight_decay))(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)
```

Типичный FCN FCN_32s из первого попавшегося репозитория на гитхабе.

1. Блоки со свертками по:
    * 64
    * 128
    * 256
    * 512
    * 512

2. Полносвязный слой(ну как бы полносвязный, часто заменяется свертками 1 * 1)
3. BilinearUpsampling2D (:и так сойдет:) - но можно заменить и на Conv2DTranspose

Вариации и их работа:
<img src='imgs/fcn_variations.png'/>

## Другой репозиторий
https://github.com/veugene/fcn_maker

Вообще конструктор разнообразных FCN|VNET|UNET


Блоки(общие для более-менее всех сетей):
1. Preprocessor
2. Encoder blocks
3. Bottleneck (across blocks)
4. Decoder blocks
5. Postprocessor


# UNET
<img src='imgs/unet.png'/>

<img src='imgs/semantic_segmentation.png'/>

# Отлично справляется с:
1. спутниковыми снимками
2. медициной
3. машинами
4. кажется, что да вообще со всем

<img src='imgs/unet_carvana.png'/>

# Функции потерь

* BCE и Weighted BCE loss
    * $\LARGE CE(p, \hat{p}) = -(\beta p \cdot log(\hat{p})+ (1-\beta)(1-p)\cdot log(1-\hat{p}))$
    
* Focal loss
    * $\LARGE CE(p, \hat{p}) = -(\alpha (1 - \hat{p})^\gamma \cdot p \cdot log(\hat{p})+ (1-\alpha)\cdot p^\gamma \cdot(1-p)\cdot log(1-\hat{p}))$
* IOU/DICE loss
    * $\LARGE DC = \frac{2|X|\cap|Y|}{|X|+|Y|}$
    * $\LARGE IoU = \frac{|X|\cap|Y|}{|X|+|Y| - |X|\cap|Y|} = \frac{TP}{TP+FP+FN}$    

# Tiramisu
<img src='imgs/tiramisu.png'/>

# PSP Net
<img src='imgs/psp_net.png'/>

# SOURCE

cs231n

http://warmspringwinds.github.io/tensorflow/tf-slim/2016/11/22/upsampling-and-image-segmentation-with-tensorflow-and-tf-slim/

https://medium.com/beyondminds/a-simple-guide-to-semantic-segmentation-effcf83e7e54